In [133]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [134]:
def label_2sigma_angle(ang, mean_ang, std_ang):
    if (abs(ang) > (mean_ang + 2.0 * std_ang)):
        return True
    else:
        return False

def label_3sigma_trq(trq, mean_trq, std_trq):
    if (trq > (mean_trq + 3.0 * std_trq)):
        return True
    else:
        return False

def calc_point2point(actual, predict):
    """
    calculate f1 score by predict and actual.
    Args:
        actual (np.ndarray): np.ndarray
        predict (np.ndarray): the predict label
    """
    TP = np.sum(predict * actual)
    TN = np.sum((1 - predict) * (1 - actual))
    FP = np.sum(predict * (1 - actual))
    FN = np.sum((1 - predict) * actual)
    precision = TP / (TP + FP + 0.00001)
    recall = TP / (TP + FN + 0.00001)
    f1 = 2 * precision * recall / (precision + recall + 0.00001)
    return f1, precision, recall, TP, TN, FP, FN

In [135]:
df = pd.read_csv('anomaly.csv')

labels = np.zeros(len(df)).astype(int)
labels = pd.DataFrame(labels, columns=['labels'])
print(labels.head())

mean_ang = np.mean(df['ANG'], axis=0)
std_ang = np.std(df['ANG'], axis=0)
angle_df = pd.DataFrame(df[['ANG']].apply(lambda df: label_2sigma_angle(df['ANG'], mean_ang, std_ang), axis=1), columns=['ang'])

mean_trq = np.mean(df['TRQ'], axis=0)
std_trq = np.std(df['TRQ'], axis=0)
trq_df = pd.DataFrame(df[['TRQ']].apply(lambda df: label_3sigma_trq(df['TRQ'], mean_trq, std_trq), axis=1), columns=['trq'])

anomaly = pd.concat([angle_df, trq_df], axis=1)
anomaly = np.sum(anomaly, axis=1) >= 1 + 0

anomaly = anomaly.to_frame()
anomaly.columns = ['true_anomaly']
anomaly['pred_anomaly'] = df['Anomaly']

print(df.head())

anomaly = df['Anomaly']
anomaly.to_csv('anomaly_labels.csv', index=0)

print(calc_point2point(anomaly['true_anomaly'], anomaly['pred_anomaly']))

   labels
0       0
1       0
2       0
3       0
4       0
          ANG        TRQ    ANG_pred   TRQ pred       MAE  Anomaly
0  -91.000010  2687.0000  -90.761900  2687.1611  0.000791    False
1  -88.999990  2604.9998  -87.947105  2604.1638  0.003688    False
2 -101.999990  2843.0000 -101.471200  2839.6775  0.006311    False
3  -45.000000  2443.0000  -45.590237  2443.1714  0.001611    False
4  -46.999992  2441.0000  -47.537178  2441.2737  0.001647    False


KeyError: 'true_anomaly'

In [ ]:
plt.plot(df['MAE'])